# 🧠 Engineering Note: I2C & Driver Anatomy

## The Problem
We read the Internal Temp (ADC) and the Photoresistor (ADC). These are **Analog**.
But professional sensors (like the AHT10, BME280, or OLED Screens) are **Digital**. They don't send a voltage; they send bytes.

## The Solution: I2C (Inter-Integrated Circuit)
I2C is a 2-wire bus protocol. It allows one "Master" (Pico) to talk to 127 "Slaves" (Sensors) using just two pins.

### 1. The Wires
- **SDA (Serial Data)**: Send/Receive bits (0 or 1).
- **SCL (Serial Clock)**: The ticker that keeps time (Wait for the tick to read the bit).

### 2. The Address (0xHH)
Every sensor has a unique ID, like a house address. The Pico yells: "Hey House **0x3C**!" (OLED). Everyone else ignores it.

### 3. Registers (The Mailboxes)
Inside the sensor, data is stored in "Registers".
- Register `0x00`: Might hold the Device ID.
- Register `0x33`: Might hold the Temperature.

## Anatomy of a Python Driver
When you download a library (e.g. `ssd1306.py`), you are just using a wrapper around 3 steps:

```python
class MySensor:
    def __init__(self, i2c, address=0x38):
        self.i2c = i2c
        self.address = address
        
    def read_temperature(self):
        # 1. Ask for data (Write command to Register 0xAC)
        self.i2c.writeto(self.address, bytes([0xAC]))
        
        # 2. Reading the response (Read 6 bytes)
        # The sensor needs time to measure, so we might sleep
        data = self.i2c.readfrom(self.address, 6)
        
        # 3. Bit Shifting (Math to combine bytes)
        # Temp is usually split across multiple bytes
        raw_temp = (data[3] & 0x0F) << 16 | data[4] << 8 | data[5]
        return raw_temp
```

In Phase 5, we will use I2C to control the OLED Screen.